In [1]:
# 필요한 라이브러리 설치
!pip install faiss-cpu
!pip install sentence-transformers
!pip install transformers
!pip install fastapi uvicorn nest_asyncio pyngrok
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 12.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nv

In [2]:
# ================================
# 1. Sentence-BERT 임베딩
# ================================
from fastapi import FastAPI, Request
from pydantic import BaseModel
from typing import List
import numpy as np
import faiss

from sentence_transformers import SentenceTransformer
from transformers import pipeline, GPT2Tokenizer, GPTNeoForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

app = FastAPI()

# Sentence-BERT 모델 로드
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # 문장 임베딩 특화

# GPT-Neo 모델 로드
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer.pad_token = tokenizer.eos_token
qa_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Device set to use cuda:0


In [3]:
from datasets import Dataset
import torch
import json
from google.colab import files

uploaded = files.upload()

# 2. 데이터 로딩 (예: 질문+문맥 => 정답)
# JSONL 또는 리스트 형식 예시:
# [
#   {"question": "BERT는 무엇인가요?", "context": "BERT는 ...", "answer": "BERT는 자연어 처리 모델입니다."}
# ]

with open("qa_finetune_data.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

Saving qa_finetune_data.json to qa_finetune_data.json


In [4]:
# 3. 학습용 포맷 구성
formatted_data = []
for item in raw_data:
    prompt = f"질문: {item['Question']}\n"
    completion = f"답변: {item['Answer']}"
    formatted_data.append({"text": prompt + completion})

In [5]:
# 4. Dataset 변환
dataset = Dataset.from_list(formatted_data)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

In [6]:
# 5. 학습 설정
training_args = TrainingArguments(
    output_dir="./gptneo-finetuned-qa",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    logging_steps=20,
    save_steps=200,
    save_total_limit=1,
    fp16=torch.cuda.is_available(),
)

In [7]:
# 6. Trainer 구성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-7-9a61c171796e>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
# 7. 학습 시작
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: circlehalf17 (circlehalf17-no-job) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
20,0.816000


TrainOutput(global_step=30, training_loss=0.6753046592076619, metrics={'train_runtime': 99.6332, 'train_samples_per_second': 0.632, 'train_steps_per_second': 0.301, 'total_flos': 215317708013568.0, 'train_loss': 0.6753046592076619, 'epoch': 2.761904761904762})

In [9]:
# 8. 모델 저장
model.save_pretrained("/content/drive/MyDrive/gptneo-finetuned-qa")
tokenizer.save_pretrained("/content/drive/MyDrive/gptneo-finetuned-qa")

('/content/drive/MyDrive/gptneo-finetuned-qa/tokenizer_config.json',
 '/content/drive/MyDrive/gptneo-finetuned-qa/special_tokens_map.json',
 '/content/drive/MyDrive/gptneo-finetuned-qa/vocab.json',
 '/content/drive/MyDrive/gptneo-finetuned-qa/merges.txt',
 '/content/drive/MyDrive/gptneo-finetuned-qa/added_tokens.json')

In [2]:
# ================================
# 1. Sentence-BERT 임베딩
# ================================
from fastapi import FastAPI, Request
from pydantic import BaseModel
from typing import List
import numpy as np
import faiss

from sentence_transformers import SentenceTransformer
from transformers import pipeline, GPT2Tokenizer, GPTNeoForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

app = FastAPI()

# Sentence-BERT 모델 로드
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # 문장 임베딩 특화

# GPT-Neo 모델 로드
model = GPTNeoForCausalLM.from_pretrained("/content/drive/MyDrive/gptneo-finetuned-qa")
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/gptneo-finetuned-qa")
tokenizer.pad_token = tokenizer.eos_token
qa_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [3]:
def get_bert_embedding(text):
    """
    Sentence-BERT 기반 임베딩 생성
    """
    embedding = embedding_model.encode([text], normalize_embeddings=True)  # 코사인 유사도용 정규화 포함
    return embedding.astype('float32')

In [4]:
# ================================
# 2. FAISS 인덱스 생성
# ================================
def create_faiss_index(embeddings):
    """
    FAISS 인덱스 생성 (코사인 유사도용 Inner Product)
    """
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index

In [5]:
# ================================
# 3. GPT-Neo 응답 생성 (Pipeline 사용)
# ================================
def generate_gpt_response(context, question):
    """
    GPT-Neo를 사용한 답변 생성
    """
    prompt = f"문맥: {context}\n질문: {question}\n답변:"
    response = qa_generator(
        prompt,
        max_new_tokens=500,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
        )
    answer = response[0]['generated_text'].split("답변:")[-1].strip()
    return answer

In [6]:
# Q&A 데이터셋
qa_texts = [
    "예선대회 개최 후, 결과발표 예정일이 사이트 공지사항에 안내되며 보통 예선대회 종료 후 20일정도 소요됩니다. 모든 답안을 평가위원들이 수기로 채점하기 때문에 발표일까지 다소 시간이 걸리는 점 참고하시기 바랍니다."
    "예선대회는 정답 외에도 풀이과정, 수정횟수, 제출시간 등이 모두 포함되어 순위가 결정되므로 팀별 점수와 문제는 공지되지 않습니다."
    "최종 제출한 답안과 답안을 찾은 과정 및 수정제출로 인한 감점의 합계로 채점됩니다. 동점팀이 발생한 경우에는 제출시간과 풀이과정을 재평가하여 채점됩니다."
    "대회 기간을 놓치신 경우에는 대회에 참여하실 수 없습니다. 대회는 매년 1회 개최되며, 다음 대회는 홈페이지를 통해 공지됩니다."
    "답안은 각 팀의 팀장만 작성이 가능하며, 문제보기 창에서 오른쪽 상단의 답올리기 버튼을 클릭하시면 작성이 가능합니다. 임시답안 및 최종답안 버튼을 클릭 시에는 작성한 답안 확인만 가능하며 작성이 되지 않습니다."
    "한 사람이 여러 개의 아이디를 사용하는 경우, 원서접수를 한 아이디로만 문제 확인이 가능합니다. 아닐 경우, 정상 접수가 되지 않은 경우이니 사무국으로 문의하시기 바랍니다."
    "원서접수가 되지 않은 경우 문제보기가 되지 않습니다. 사무국으로 문의하시기 바랍니다."
    "예선기간 동안 홈페이지를 통해 주어진 문제를 팀원들끼리 온라인 또는 오프라인 통해 함께 해결하시면 됩니다. 문제 확인은 팀원 모두가 확인할 수 있으며, 답안 작성은 팀장만 가능합니다. 기간 내 답안을 여러 번 수정 제출할 수 있지만, 수정 시 1회마다 1점씩 감점됩니다. 답안은 최종 제출한 답안으로 처리됩니다."
    "본 대회에서 제공하는 예선 및 본선문제, 학습실 제공 문제 등은 대회 운영사무국의 창작 저작물인 관계로 타 사이트, 블로그 및 기타 SNS, 문서 등으로 원본 또는 수정 형태로 절대 배포가 불가합니다. 문제가 비공개로 진행되는 대회 특성상, 사전허가 없이 전재·방송하거나 무단으로 복사·배포·판매·전시·개작할 경우 민·형사상 책임이 따를 수 있습니다."
    "신청 마감일에 가까울수록 접속자가 몰려 사이트 접속이 어려울 수 있으니 꼭 미리 접수하시기 바랍니다. 마감일 접속자 폭주로 인해 정상접수가 되지 않은 경우에도 추가 접수는 절대 불가합니다."
    "원서 접수 기간 이후 추가 접수는 절대 불가합니다. (신청 마감일에 가까울수록 접속자가 몰려 사이트 접속이 어려울 수 있으니 미리 접수하시기 바랍니다. 마감일 접속자 폭주로 인한 정상접수가 되지 않은 경우에도 추가 접수는 절대 불가합니다. 단, 사무국에서 기간 연장을 전체 공지한 경우는 연장 기간까지 접수 가능)"
    "나이가 초, 중, 고등생과 같더라도 재학생이 아닌 경우에는 대학일반부로 참가하셔야합니다."
    "해외거주(재학)중에도 회원가입은 가능하며, 예선(온라인 개최) 및 본선(오프라인 개최)대회에 참가 가능하시면 누구나 대회 참가하실 수 있습니다. 단, 마감기한은 한국시간을 기준으로 하며 시차 등 해외 거주 특성상 원서접수, 대회 참가가 정상적으로 이루어지지 않는 경우에는 본 대회운영 사무국에서는 책임지지 않습니다."
    "팀명을 7자 이상 너무 길게 작성하시면, 오류로 인해 정상 접수가 되지 않을 수도 있습니다."
    "사이트의 경진대회 > 참가신청 > 접수확인에서 확인가능하며, 확인이 되지 않을 경우 사무국으로 직접 문의하시기 바랍니다."
    "대회에 참가하는 팀원이 모두 대회 사이트(www.cpsfestival.org)에 회원가입을 해야 합니다. 단, 대회 참가신청은 팀장이 대표로 원서접수 하시면 되며 대회 원서접수 페이지에서 팀장 및 팀원의 아이디와 비밀번호를 입력해야 팀원이 모두 대회에 참가하실 수 있습니다."
    "다른 부문끼리는 팀 구성이 불가합니다. (예, 초등학생-고등학생 팀 구성 등)"
    "같은 학교 학생이 아니더라도 같은 부문끼리면 팀 구성 가능합니다."
    "전국 초, 중, 고, 대학/일반 (해외거주 및 재학생 포함) 누구나 참가 가능합니다. 단, 같은 부문(초, 중, 고, 대학/일반)끼리 2인~3인이 팀 구성을 해야 하며 대학일반부는 팀별 여성참가자 1인 필수입니다."
    "대회 개최 안내는 매년 5월 홈페이지를 통해 공고됩니다. 일반적으로 원서접수는 6월 중순 ~ 7월 중순, 예선대회는 7월 말, 본선대회는 8월 말, 시상식은 11월에 개최되고 있습니다. 일정은 개최 시기에 따라 변동될 수 있습니다."
    "관련 문의사항은 사이트의 게시판이나 사무국으로 직접 문의하시면 됩니다."
]

embeddings = np.vstack([get_bert_embedding(text) for text in qa_texts])
index = create_faiss_index(embeddings)

# --------------------------
# 입력/출력 모델 정의
# --------------------------
class QARequest(BaseModel):
    question: str
    top_k: int = 3

class QAResponse(BaseModel):
    question: str
    response: str
    context: List[str]

# --------------------------
# FastAPI 라우터
# --------------------------
@app.post("/qa", response_model=QAResponse)
def qa_endpoint(request: QARequest):
    question = request.question
    top_k = request.top_k

    # 질문 임베딩
    query_vec = get_bert_embedding(question)
    distances, indices = index.search(query_vec, top_k)
    print(distances)
    print(indices)

    # 유사 문맥 추출
    similar_texts = [qa_texts[i] for i in indices[0]]
    context = " ".join(similar_texts)

    # GPT 응답 생성
    gpt_response = generate_gpt_response(context, question)

    return QAResponse(
        question=question,
        response=gpt_response,
        context=similar_texts
    )

In [7]:
# 2. ngrok 실행 코드
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# 3. ngrok 설정
ngrok.set_auth_token("2v1Fi5CEzLumREBpheNMIIepRlM_7uLFbq5PGe81hmEZiAe9K")  # 🔑 Ngrok 토큰 입력 (한 번만 필요)
ngrok.kill()  # 이전 터널 종료
public_url = ngrok.connect(3000)  # 로컬 3000 포트를 외부에 노출
print("🔗 Public URL:", public_url.public_url)

# 4. 이벤트 루프 충돌 방지 (Colab 전용)
nest_asyncio.apply()

# 5. uvicorn 실행
uvicorn.run(app, host="0.0.0.0", port=3000)

INFO:     Started server process [1595]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:3000 (Press CTRL+C to quit)


🔗 Public URL: https://ec35-35-201-138-147.ngrok-free.app
INFO:     155.230.85.158:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     155.230.85.158:0 - "GET /openapi.json HTTP/1.1" 200 OK
[[0.43822706]]
[[0]]
INFO:     155.230.85.158:0 - "POST /qa HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1595]


In [ ]:
# ================================
# 4. Q&A 파이프라인
# ================================
def qa_pipeline(question, index, qa_texts, k=3):
    """
    Q&A 시스템 전체 파이프라인
    """
    # 1. 질문 임베딩
    query_vec = get_bert_embedding(question)

    # 2. FAISS 검색
    distances, indices = index.search(query_vec, k)

    # 3. 상위 k개의 유사 문장 추출
    similar_texts = [qa_texts[i] for i in indices[0]]

    # 4. 문맥 조합
    context = " ".join(similar_texts)

    # 5. GPT로 응답 생성
    gpt_response = generate_gpt_response(context, question)

    # 🔍 로그 출력
    print("📌 질문:", question)
    print("🔎 검색된 문장:")
    for i, text in enumerate(similar_texts, 1):
        print(f"{i}. {text}")
    print("🧠 GPT 응답:", gpt_response)

    return gpt_response

In [ ]:
# ================================
# 5. 실행 테스트
# ================================
# Q&A 데이터셋
qa_texts = [
    "AI는 인공지능의 줄임말로, 기계가 사람처럼 학습하고 추론할 수 있도록 하는 기술입니다.",
    "FAISS는 Facebook AI에서 만든 고차원 벡터 검색 라이브러리입니다.",
    "BERT는 자연어 이해를 위한 사전 훈련된 트랜스포머 모델입니다.",
    "GPT-neo는 언어 생성을 위한 트랜스포머 모델로, 대규모 데이터를 바탕으로 훈련되었습니다.",
    "코사인 유사도는 두 벡터의 방향성을 비교해 얼마나 유사한지 측정하는 지표입니다."
]

# 문장 임베딩 벡터 생성
embeddings = np.vstack([get_bert_embedding(text) for text in qa_texts])

# FAISS 인덱스 생성
index = create_faiss_index(embeddings)

# 테스트 질문
question = "인공지능이 뭐야?"

# 응답 생성
response = qa_pipeline(question, index, qa_texts)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


📌 질문: 인공지능이 뭐야?
🔎 검색된 문장:
1. AI는 인공지능의 줄임말로, 기계가 사람처럼 학습하고 추론할 수 있도록 하는 기술입니다.
2. 코사인 유사도는 두 벡터의 방향성을 비교해 얼마나 유사한지 측정하는 지표입니다.
3. BERT는 자연어 이해를 위한 사전 훈련된 트랜스포머 모델입니다.
🧠 GPT 응답: 전투에 사물이 취약점에 따라 있다. 유사도 전투를 통해 해석한 반응한 훈련을 통해 그 전과 연기와 연기를 시작한 반응훈련을 써도 연결하여 이어야 합니다.
“전과 연�
response:  전투에 사물이 취약점에 따라 있다. 유사도 전투를 통해 해석한 반응한 훈련을 통해 그 전과 연기와 연기를 시작한 반응훈련을 써도 연결하여 이어야 합니다.
“전과 연�
